# MNIST and NN model insterpretation?

In this exercise we will work with a full size version of the MNIST dataset of handwritten digits.

In [1]:
import tensorflow as tf
color_chart = ["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]

(X_train, y_train), (X_val, y_val) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 7s 1us/step


* Check out the shape of X_train

In [2]:
X_train.shape

(60000, 28, 28)

* Visualize one element from this dataset

In [3]:
from plotly import express as px
px.imshow(X_train[0], color_continuous_scale="gray")

* This data shape won't fit in a fully connected neural network, reshape the elements from the dataset so they are flat.

In [4]:
X_train = X_train.reshape([X_train.shape[0],-1])
X_val = X_val.reshape([X_val.shape[0],-1])

X_train.shape

(60000, 784)

* The data's training variables are pixels which values range from 0 to 255, divide them by 255 so they are normalized

In [5]:
X_train = X_train/255
X_val = X_val/255

* Convert the train and test into tensor datasets

In [6]:
train = tf.data.Dataset.from_tensor_slices((X_train,y_train))
val = tf.data.Dataset.from_tensor_slices((X_val,y_val))
train

<TensorSliceDataset element_spec=(TensorSpec(shape=(784,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>

* Suffle the datasets and organise them by batches of 16

In [7]:
train_batch = train.shuffle(len(train)).batch(16)
val_batch = val.shuffle(len(val)).batch(16)

next(iter(train_batch))

(<tf.Tensor: shape=(16, 784), dtype=float64, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])>,
 <tf.Tensor: shape=(16,), dtype=uint8, numpy=array([9, 3, 0, 2, 1, 7, 3, 7, 2, 5, 9, 9, 7, 3, 0, 9], dtype=uint8)>)

* What is the effect of the batch size on the training of the model? If the batch size is equal to the number of observations, what would the batch gradient descent be equivalent to?
* With batches of 16 observations, how many times will the parameters of the model be updated before we reach one epoch?

* Create a simple neural network model, don't forget to set an input shape in the first layer that corresponds to the data we will feed the model and define the last layer so it is able to make predictions as we expect.

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

model = Sequential([
                    Dense(16,"relu",input_shape=[X_train.shape[1]]),
                    Dense(8,"relu"),
                    Dense(len(np.unique(y_train)),"softmax")
])

tf.keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


* Print your model's summary, can you understand the number of parameters on each layer?

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                12560     
                                                                 
 dense_1 (Dense)             (None, 8)                 136       
                                                                 
 dense_2 (Dense)             (None, 10)                90        
                                                                 
Total params: 12,786
Trainable params: 12,786
Non-trainable params: 0
_________________________________________________________________


* Compile the model using the right loss and metric and the Adam optimizer

In [10]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

model.compile(optimizer=Adam(0.001),
              loss=SparseCategoricalCrossentropy(),
              metrics=[SparseCategoricalAccuracy()])

* Train the model on 10 epochs

In [11]:
history = model.fit(train_batch,epochs=10,validation_data=val_batch)

Epoch 1/10
3750/3750 [==============================] - 3s 784us/step - loss: 0.4602 - sparse_categorical_accuracy: 0.8628 - val_loss: 0.2615 - val_sparse_categorical_accuracy: 0.9244
Epoch 2/10
3750/3750 [==============================] - 3s 748us/step - loss: 0.2414 - sparse_categorical_accuracy: 0.9307 - val_loss: 0.2176 - val_sparse_categorical_accuracy: 0.9368
Epoch 3/10
3750/3750 [==============================] - 3s 747us/step - loss: 0.2039 - sparse_categorical_accuracy: 0.9410 - val_loss: 0.2022 - val_sparse_categorical_accuracy: 0.9405
Epoch 4/10
3750/3750 [==============================] - 3s 743us/step - loss: 0.1816 - sparse_categorical_accuracy: 0.9470 - val_loss: 0.1843 - val_sparse_categorical_accuracy: 0.9478
Epoch 5/10
3750/3750 [==============================] - 3s 766us/step - loss: 0.1669 - sparse_categorical_accuracy: 0.9503 - val_loss: 0.1788 - val_sparse_categorical_accuracy: 0.9496
Epoch 6/10
3750/3750 [==============================] - 3s 752us/step - loss: 0.

* Would you say the model achieved good performance? Did it overfit?

In [12]:
from plotly import graph_objects as go
fig = go.Figure(data=[
                      go.Scatter(
                          y=history.history["loss"],
                          name="Training loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[0]
                          )),
                      go.Scatter(
                          y=history.history["val_loss"],
                          name="Validation loss",
                          mode="lines",
                          marker=dict(
                              color=color_chart[1]
                          ))
])
fig.update_layout(
    title='Training and val loss across epochs',
    xaxis_title='epochs',
    yaxis_title='Cross Entropy'    
)
fig.show()

* Get the weights from the first layer

In [13]:
first_layer_weights = model.layers[0].kernel
first_layer_weights

<tf.Variable 'dense/kernel:0' shape=(784, 16) dtype=float32, numpy=
array([[-0.01868633,  0.01162615,  0.06739259, ...,  0.04333854,
         0.07438236,  0.00403195],
       [ 0.0576276 ,  0.07825045,  0.03938662, ..., -0.04926601,
         0.08348103,  0.04503697],
       [ 0.04177362, -0.07995173, -0.0651654 , ...,  0.07571323,
         0.04488543, -0.03258891],
       ...,
       [ 0.01949845, -0.07834168, -0.04564701, ..., -0.08569675,
        -0.00727006,  0.03317396],
       [ 0.0129412 , -0.06831755,  0.06292443, ...,  0.00132391,
        -0.04200315, -0.08646984],
       [ 0.07282227, -0.01081819, -0.06019029, ...,  0.0290183 ,
         0.07781573, -0.01928524]], dtype=float32)>

* Extract the weights from one of the neurons on that layer, what does the number of weights correspond to?

In [14]:
first_neuron_first_layer_weights = first_layer_weights.numpy()[:,0]
first_neuron_first_layer_weights.shape

(784,)

* Reshape the weights to form a 28x28 image and display it, can you interpret the weights on this neuron?

In [15]:
px.imshow(first_neuron_first_layer_weights.reshape([28,28]), color_continuous_scale="gray")

* Try the same thing with all the neurons of the first layer, can you see something interpretable? * Try the same thing with all the neurons of the first layer, can you see something interpretable? Youcan find help <a href="https://plotly.com/python/heatmaps/"> here </a> and <a href="https://plotly.com/python/subplots/"> here </a>.

In [16]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=first_layer_weights.shape[1]//4, cols=4)
for i in range(first_layer_weights.shape[1]):
  row = i//4
  col = i - i//4*4
  neuron = first_layer_weights.numpy()[:,i]
  img = neuron.reshape([28,28])
  fig.add_trace(go.Heatmap(z=img, colorscale="gray"), row=row+1, col=col+1)

fig.update_layout(height=16*400/4, width=1200)

fig.show()

* Do the same visualization but apply relu to the weights

In [17]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=first_layer_weights.shape[1]//4, cols=4)
for i in range(first_layer_weights.shape[1]):
  row = i//4
  col = i - i//4*4
  neuron = tf.keras.activations.relu(first_layer_weights.numpy()[:,i]).numpy()
  img = neuron.reshape([28,28])
  fig.add_trace(go.Heatmap(z=img, colorscale="gray"), row=row+1, col=col+1)

fig.update_layout(height=16*400/4, width=1200)

fig.show()

* The parameters on each of the neurons of the first layer let's you see what information the first layer of the network is taking from the data to create new features, it would also be possible to analyse the features from the second layer by combining those parameters thanks to the weights of the second layer.

In [18]:
first_layer_w = model.layers[0].kernel
first_layer_b = model.layers[0].bias

second_layer_w = model.layers[1].kernel
second_layer_b = model.layers[1].bias

In [19]:
second_layer_feature = np.dot(tf.keras.activations.relu(first_layer_b + first_layer_w).numpy(),
                              second_layer_w[:,3].numpy()).reshape([28,28])
px.imshow(second_layer_feature, color_continuous_scale="gray")

* Apply relu to this new level of feature to see what goes to the last layer

In [20]:
second_layer_output = tf.keras.activations.relu(second_layer_feature).numpy()
px.imshow(second_layer_output, color_continuous_scale="gray")

* Bottomline is it's quite hard to understand how the model is making predictions, from what we see it's picking up some details about borders or contrasts at the edge of numbers, but it does not seem that the model is percieving numbers the way our brain does!
* Do not worry though, tensorflow has tools that make it easy to see what elements of the images are used to make the predictions, we will see this in what follows.